<a href="https://colab.research.google.com/github/talktovishal/NLP-ML-Projects/blob/master/Project2_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
!pip install kaggle


In [0]:
!mkdir /root/.kaggle


In [0]:
!echo '{"username":"thevishalchowdhary","key":"0149faaacbbf52163a7ef279cd3264e3"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
import json
import zipfile
import os

In [0]:
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
sudalairajkumar/novel-corona-virus-2019-dataset             Novel Corona Virus 2019 Dataset                     329KB  2020-03-05 06:14:42          19809  
kimjihoo/coronavirusdataset                                 Coronavirus-Dataset                                  20KB  2020-03-05 01:57:50           4134  
rupals/gpu-runtime                                          Segmentation GPU Kernel Performance Dataset           4MB  2020-03-01 10:04:27             56  
anlthms/dfdc-video-faces                                    DFDC video face crops - parts 4-8                     2GB  2020-03-03 01:22:58             19  
prakrutchauhan/indian-candidates-for-general-election-2019  Indi

In [0]:
!kaggle datasets download snapcrack/all-the-news

 98% 238M/244M [00:01<00:00, 131MB/s]
100% 244M/244M [00:01<00:00, 137MB/s]


In [0]:
!os.chdir('/content')
for file in os.listdir():
    print(file)
    if(file.endswith('.zip')):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

/bin/bash: -c: line 0: syntax error near unexpected token `'/content''
/bin/bash: -c: line 0: `os.chdir('/content')'
.config
all-the-news.zip
sample_data


In [0]:
!pwd

/content


In [0]:
!cd sample_data

In [0]:
!ls -a -l

total 903348
drwxr-xr-x 1 root root      4096 Mar  5 07:07 .
drwxr-xr-x 1 root root      4096 Mar  5 06:57 ..
-rw-r--r-- 1 root root 255356300 Mar  5 07:07 all-the-news.zip
-rw-r--r-- 1 root root 203539364 Mar  5 07:07 articles1.csv
-rw-r--r-- 1 root root 225757056 Mar  5 07:07 articles2.csv
-rw-r--r-- 1 root root 240344348 Mar  5 07:07 articles3.csv
drwxr-xr-x 1 root root      4096 Mar  3 18:11 .config
drwxr-xr-x 1 root root      4096 Mar  3 18:11 sample_data


In [0]:
import nltk
#for stop words
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import string

lemmatizer = WordNetLemmatizer()

#lemmatizer code from https://simonhessner.de/lemmatize-whole-sentences-with-python-and-nltks-wordnetlemmatizer/
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
def nltk2wn_tag(nltk_tag):
  if nltk_tag.startswith('J'):
    return wordnet.ADJ
  elif nltk_tag.startswith('V'):
    return wordnet.VERB
  elif nltk_tag.startswith('N'):
    return wordnet.NOUN
  elif nltk_tag.startswith('R'):
    return wordnet.ADV
  else:          
    return None

def lemmatize_sentence(sentence):
  nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
  wn_tagged = map(lambda x: (x[0], nltk2wn_tag(x[1])), nltk_tagged)
  res_words = []
  for word, tag in wn_tagged:
    if tag is None:            
      res_words.append(word)
    else:
      res_words.append(lemmatizer.lemmatize(word, tag))
  return res_words

def normalizeSentence(sentence):
  #lowercase
  updatedSentence = sentence.lower()
  #remove punctuations
  updatedSentence = updatedSentence.translate(str.maketrans('','',string.punctuation))
  return updatedSentence


#not i am not removing stop words since i plan to use this for tf-idf calculation
def normalizeAndLemmatizeSentence(sentence):
  #lowercase
  updatedSentence = sentence.lower()
  #remove punctuations
  updatedSentence = updatedSentence.translate(str.maketrans('','',string.punctuation))
  #lemmatization
  return lemmatize_sentence(updatedSentence)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
import nltk.data
nltk.download('punkt')
sentence_detection = nltk.data.load('tokenizers/punkt/english.pickle')
#https://www.nltk.org/api/nltk.tokenize.html
def extract_sentences(text):
  return sentence_detection.tokenize(text)


def extract_n_normalize_sentences(text):
  allSentences = sentence_detection.tokenize(text)
  updatedSentences = []
  for sentence in allSentences:
    #NOTE: You need an additional [] bracket before you add content to the list
    #Reason = https://stackoverflow.com/questions/29947007/array-extendstring-adds-every-character-instead-of-just-the-string
    updatedSentences.extend([normalizeSentence(sentence)])
  return updatedSentences


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
normalizedSentences = []
for file in os.listdir():
    if(file.endswith('.csv')):
      pdf = pd.read_csv(file)
      print(f'Read file {file} into a panda dataframe')
      
      #this is a very slow method to iterate over the list and apply the normlize function.
      #do somethign better: https://chrisalbon.com/python/basics/applying_functions_to_list_items/
      #pdf['content'].apply(lambda content: normalizedSentences.extend(extract_n_normalize_sentences(content)))

      pdf['content'].apply(lambda content: normalizedSentences.extend([normalizeSentence(s) for s in extract_sentences(content)]))
      #pdf.info()
      #pdf.head(3)

Read file articles2.csv into a panda dataframe
Read file articles3.csv into a panda dataframe
Read file articles1.csv into a panda dataframe


In [0]:
# pdf = pd.read_csv('articles1.csv')
# print(f'Read file articles1.csv into a panda dataframe')
# pdf.info()
# pdf.head(3)


In [0]:
# pdf['content'][0]
# normalizedSentences = []
# pdf['content'].apply(lambda content: normalizedSentences.extend([content]))

In [0]:
normalizedSentences[6749]

'will trump have the same patience as his generals'

In [0]:
len(normalizedSentences)

5070143

In [0]:
len(pdf)

50000

In [0]:
#to make everything else run faster during testing\development.
normalizedSentences = normalizedSentences[:50000]

In [0]:
len(normalizedSentences)


50000

In [0]:
%matplotlib inline
#implement CBOW using pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#to have re-producable results
torch.manual_seed(1)

In [0]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

In [0]:
from nltk.tokenize import word_tokenize
wordsInSentence = [word_tokenize(s) for s in normalizedSentences]

In [0]:
#get the vocab
vocab = set()
for words in wordsInSentence:
  vocab.update(words)


In [0]:
len(vocab)

36191

In [0]:
import random
for i, val in enumerate(random.sample(vocab, 10)):
  print(val)

chairman
merger
reserves
moses
thinly
reprinted
crystal
outlook
prominence
minor


In [0]:
vocab_size = len(vocab)

In [0]:
word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for raw_text in wordsInSentence:
  for i in range(2, len(raw_text) - 2):
      context = [
                 word_to_ix[raw_text[i - 2]], 
                 word_to_ix[raw_text[i - 1]],
                 word_to_ix[raw_text[i + 1]], 
                 word_to_ix[raw_text[i + 2]]
                 ]
      target = word_to_ix[raw_text[i]]
      data.append((context, target))

print(data[:5])


[([24417, 24305, 17752, 19501], 31045), ([24305, 31045, 19501, 14181], 17752), ([31045, 17752, 14181, 30909], 19501), ([17752, 19501, 30909, 1664], 14181), ([19501, 14181, 1664, 1538], 30909)]


In [0]:
len(data)

1000831

In [0]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)  # example

KeyError: ignored